# Model Comparison with MLJFlux

In this workflow example, we see how we can compare different machine learning models
with a neural network from MLJFlux.

In [1]:
using Pkg
Pkg.activate(@__DIR__);
Pkg.instantiate();

  Activating project at `~/GoogleDrive/Julia/MLJ/MLJFlux/docs/src/workflow examples/Comparison`


**Julia version** is assumed to be 1.10.*

### Basic Imports

In [2]:
using MLJ               # Has MLJFlux models
using Flux              # For more flexibility
import RDatasets        # Dataset source
using DataFrames        # To visualize hyperparameter search results
import Optimisers       # native Flux.jl optimisers no longer supported

### Loading and Splitting the Data

In [3]:
iris = RDatasets.dataset("datasets", "iris");
y, X = unpack(iris, ==(:Species), colname -> true, rng=123);

### Instantiating the models Now let's construct our model. This follows a similar setup
to the one followed in the [Quick Start](../../index.md#Quick-Start).

In [4]:
NeuralNetworkClassifier = @load NeuralNetworkClassifier pkg=MLJFlux

clf1 = NeuralNetworkClassifier(
    builder=MLJFlux.MLP(; hidden=(5,4), σ=Flux.relu),
    optimiser=Optimisers.Adam(0.01),
    batch_size=8,
    epochs=50,
    rng=42
    )

[ Info: For silent loading, specify `verbosity=0`. 
import MLJFlux ✔


NeuralNetworkClassifier(
  builder = MLP(
        hidden = (5, 4), 
        σ = NNlib.relu), 
  finaliser = NNlib.softmax, 
  optimiser = Adam(0.01, (0.9, 0.999), 1.0e-8), 
  loss = Flux.Losses.crossentropy, 
  epochs = 50, 
  batch_size = 8, 
  lambda = 0.0, 
  alpha = 0.0, 
  rng = 42, 
  optimiser_changes_trigger_retraining = false, 
  acceleration = CPU1{Nothing}(nothing))

Let's as well load and construct three other classical machine learning models:

In [5]:
BayesianLDA = @load BayesianLDA pkg=MultivariateStats
clf2 = BayesianLDA()
RandomForestClassifier = @load RandomForestClassifier pkg=DecisionTree
clf3 = RandomForestClassifier()
XGBoostClassifier = @load XGBoostClassifier pkg=XGBoost
clf4 = XGBoostClassifier();

[ Info: For silent loading, specify `verbosity=0`. 
import MLJMultivariateStatsInterface ✔
[ Info: For silent loading, specify `verbosity=0`. 
import MLJDecisionTreeInterface ✔
[ Info: For silent loading, specify `verbosity=0`. 
import MLJXGBoostInterface ✔


### Wrapping One of the Models in a TunedModel Instead of just comparing with four
models with the default/given hyperparameters, we will give `XGBoostClassifier` an
unfair advantage By wrapping it in a `TunedModel` that considers the best learning rate
η for the model.

In [6]:
r1 = range(clf4, :eta, lower=0.01, upper=0.5, scale=:log10)
tuned_model_xg = TunedModel(
    model=clf4,
    ranges=[r1],
    tuning=Grid(resolution=10),
    resampling=CV(nfolds=5, rng=42),
    measure=cross_entropy,
);

Of course, one can wrap each of the four in a TunedModel if they are interested in
comparing the models over a large set of their hyperparameters.

### Comparing the models
We simply pass the four models to the `models` argument of the `TunedModel` construct

In [7]:
tuned_model = TunedModel(
    models=[clf1, clf2, clf3, tuned_model_xg],
    tuning=Explicit(),
    resampling=CV(nfolds=5, rng=42),
    measure=cross_entropy,
);

Then wrapping our tuned model in a machine and fitting it.

In [8]:
mach = machine(tuned_model, X, y);
fit!(mach, verbosity=0);

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(4 => 5, relu)  # 25 parameters
│   summary(x) = "4×8 Matrix{Float64}"
└ @ Flux ~/.julia/packages/Flux/Wz6D4/src/layers/stateless.jl:60


Now let's see the history for more details on the performance for each of the models

In [9]:
history = report(mach).history
history_df = DataFrame(
    mlp = [x[:model] for x in history],
    measurement = [x[:measurement][1] for x in history],
)
sort!(history_df, [order(:measurement)])

Row,mlp,measurement
,Probabil…,Float64
1,"BayesianLDA(method = gevd, …)",0.0610826
2,"NeuralNetworkClassifier(builder = MLP(hidden = (5, 4), …), …)",0.0857014
3,"RandomForestClassifier(max_depth = -1, …)",0.109869
4,"ProbabilisticTunedModel(model = XGBoostClassifier(test = 1, …), …)",0.221056


This is Occam's razor in practice.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*